<a href="https://colab.research.google.com/github/RedaElmar/Griffin/blob/master/Sentiments_Analysis_Darija_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

As requested, a minimal notebook approaching this task using transformers.

Questions: ask on the Discussion board and I'll try to answer :)

# Setup

Install the requirements and upload the files using the panel on the left.

# Load Data

The simpletransformers assumes two columns: text then label. Easy enough given our data.

In [2]:
!pip install aaransia
import pandas as pd
from aaransia import transliterate, SourceLanguageError
import re
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
import logging

In [3]:
# Load train
df = pd.read_csv('Train.csv', lineterminator='\n')
df['text'] = df['text'].astype(str) # One row has a float as the text
print(df.shape)
df.head()

(8401, 3)


,ID,text,label
0,IQOTJAT,m3alem bourjilia w illi ma yefehmouch yelzmou ...,1
1,HY9M63D,Ya m3alllam,1
2,7SV55S2,Ma7lek! Zin ou fannena 7loua.,1
3,KDLJVZR,hhhhh ya bliiiiiiiida ya Hanen,0
4,0MAU5GY,Nikraha w ma5yebha pffff,0


In [4]:
EMOTICONS = {
    u":)":"Happy face or smiley",
    u":D":"Happy face or smiley",
    u"<3":"Happy face or smiley",
    u":*":"Happy face or smiley",
    u":p":"Happy face or smiley",
    u":P":"Happy face or smiley",
    u"xD":"Happy face or smiley",
    u"XD":"Happy face or smiley",
    u":â€‘\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":â€‘D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8â€‘D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"Xâ€‘D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":â€‘\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":â€‘c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":â€‘<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":â€‘\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'â€‘\(":"Crying",
    u":'\(":"Crying",
    u":'â€‘\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"Dâ€‘':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":â€‘O":"Surprise",
    u":O":"Surprise",
    u":â€‘o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8â€‘0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";â€‘\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";â€‘\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":â€‘,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":â€‘P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"Xâ€‘P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":â€‘Ãž":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Ãž":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":â€‘/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":â€‘\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":â€‘x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":â€‘#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":â€‘&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:â€‘\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:â€‘3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:â€‘\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":â€‘b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:â€‘\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:â€‘\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:â€‘\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;â€‘\)":"Cool",
    u"\|â€‘O":"Bored",
    u":â€‘J":"Tongue-in-cheek",
    u"#â€‘\)":"Party all night",
    u"%â€‘\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:â€‘\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(ãƒ»\.ãƒ»;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)ï¼":"Joyful",
    u"\(\^o\^\)ï¼":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(ä¸€ä¸€\)":"Shame",
    u"\(ï¼›ä¸€_ä¸€\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\Â·\^=\)":"cat",
    u"\(=\^\Â·\Â·\^=\)":"cat",
    u"=_\^= ":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\ãƒ»\ãƒ»?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\ï¼ˆ\*\^_\^\*ï¼‰" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^â€”\^\ï¼‰":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\ï¼ˆ\^â€”\^\ï¼‰":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\Â·\Â·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ãƒ¼ãƒ¼;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\ï¼¾ï½–\ï¼¾\)":"Happy",
    u"\(\ï¼¾ï½•\ï¼¾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*ï¿£mï¿£\)":"Dissatisfied",
    u"\(â€˜A`\)":"Snubbed or Deflated"
}
emojitext=list(EMOTICONS.keys())
emojitext

[':)',
 ':D',
 '<3',
 ':*',
 ':p',
 ':P',
 'xD',
 'XD',
 ':â€‘\\)',
 ':\\)',
 ':-\\]',
 ':\\]',
 ':-3',
 ':3',
 ':->',
 ':>',
 '8-\\)',
 ':o\\)',
 ':-\\}',
 ':\\}',
 ':-\\)',
 ':c\\)',
 ':\\^\\)',
 '=\\]',
 '=\\)',
 ':â€‘D',
 '8â€‘D',
 '8D',
 'Xâ€‘D',
 '=D',
 '=3',
 'B\\^D',
 ':-\\)\\)',
 ':â€‘\\(',
 ':-\\(',
 ':\\(',
 ':â€‘c',
 ':c',
 ':â€‘<',
 ':<',
 ':â€‘\\[',
 ':\\[',
 ':-\\|\\|',
 '>:\\[',
 ':\\{',
 ':@',
 '>:\\(',
 ":'â€‘\\(",
 ":'\\(",
 ":'â€‘\\)",
 ":'\\)",
 "Dâ€‘':",
 'D:<',
 'D:',
 'D8',
 'D;',
 'D=',
 'DX',
 ':â€‘O',
 ':O',
 ':â€‘o',
 ':o',
 ':-0',
 '8â€‘0',
 '>:O',
 ':-\\*',
 ':\\*',
 ':X',
 ';â€‘\\)',
 ';\\)',
 '\\*-\\)',
 '\\*\\)',
 ';â€‘\\]',
 ';\\]',
 ';\\^\\)',
 ':â€‘,',
 ';D',
 ':â€‘P',
 'Xâ€‘P',
 'XP',
 ':â€‘Ãž',
 ':Ãž',
 ':b',
 'd:',
 '=p',
 '>:P',
 ':â€‘/',
 ':/',
 ':-[.]',
 '>:[(\\\\)]',
 '>:/',
 ':[(\\\\)]',
 '=/',
 '=[(\\\\)]',
 ':L',
 '=L',
 ':S',
 ':â€‘\\|',
 ':\\|',
 ':$',
 ':â€‘x',
 ':x',
 ':â€‘#',
 ':#',
 ':â€‘&',
 ':&',
 'O:â€‘\\)',
 'O:\\)',
 '0:â€‘3',
 '

In [5]:
!pip install emoji
import emoji

def extract_emoji(string):
  decode   = string.encode().decode('utf-8')
  allchars = [str for str in decode]
  List     = [c for c in allchars if c in emoji.UNICODE_EMOJI]
  words= string.split(' ')
  List2    = [c for c in words if c in emojitext]
  return ' '.join(List+List2)
def remove_emoji(string):
  decode   = string.encode().decode('utf-8')
  allchars = [str for str in decode]
  List     = [c for c in allchars if c in emoji.UNICODE_EMOJI]
  filtred  = [str for str in decode.split() if not any(i in str for i in List)]
  text = ' '.join(filtred)
  words= text.split(' ')
  List2    = [c for c in words if c not in emojitext]
  return ' '.join(List2)
def removeDuplicates(s):
    if s!='':
            a=s[0]
            for i in s:
                if i != a[-1]:
                    a=a+i
            return  a
    else:
        return ''
def preProcessing(tweet):    
    tweet=re.sub(r"http\S+ | www\S+" , " ", tweet)
    tweet=re.sub(r"#" , "", tweet)
    tweet=re.sub(r'@\S+',' ',tweet)
    tweet=removeDuplicates(tweet)
    return tweet
import string
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text
df['preprocessedtext'] = df['text'].apply(lambda x: preProcessing(x))
df['emojis']= df['preprocessedtext'].apply(lambda x: extract_emoji(x))
df['noemojitext'] = df['preprocessedtext'].apply(lambda x: remove_emoji(x))
df['arabictext'] = df['noemojitext'].apply(lambda x: transliterate(x, source='tn', target='ar', universal=True))
df['arabictext'] = df['arabictext'].apply(lambda x: remove_punct(x))


     |████████████████████████████████| 51kB 4.6MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=e00baac007c2a41de834281feb7ca768aeb8634588849b623140cfb97fa52eb9
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [6]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('arabic')   #all the stop words
def NoStop (txt):
  Q=[]         #checking for stop words
  a=txt.split(" ")
  for i in a:
    if i in stop or len(i)<=2:
       continue
    else:
      Q.append(i)
  return " ".join(Q)
df['NoStopWords'] = df['arabictext'].apply(NoStop)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
df.head(50)

,ID,text,label,preprocessedtext,emojis,noemojitext,arabictext,NoStopWords
0,IQOTJAT,m3alem bourjilia w illi ma yefehmouch yelzmou ...,1,m3alem bourjilia w ili ma yefehmouch yelzmou y...,,m3alem bourjilia w ili ma yefehmouch yelzmou y...,معالم بورجيليا و يلي ما يفهموش يلزمو ياقرا ان...,معالم بورجيليا يلي يفهموش يلزمو ياقرا انس پلوس...
1,HY9M63D,Ya m3alllam,1,Ya m3alam,,Ya m3alam,يا معالام,معالام
2,7SV55S2,Ma7lek! Zin ou fannena 7loua.,1,Ma7lek! Zin ou fanena 7loua.,,Ma7lek! Zin ou fanena 7loua.,ماحلك زين و فاننا حلوا,ماحلك زين فاننا حلوا
3,KDLJVZR,hhhhh ya bliiiiiiiida ya Hanen,0,h ya blida ya Hanen,,h ya blida ya Hanen,ه يا بليدا يا هانن,بليدا هانن
4,0MAU5GY,Nikraha w ma5yebha pffff,0,Nikraha w ma5yebha pf,,Nikraha w ma5yebha pf,نيكراها و ماخيبها پف,نيكراها ماخيبها
5,EA1VCJ2,3sal denya :*,1,3sal denya :*,:*,3sal denya,عسال دنيا,عسال دنيا
6,2SZ11JC,سي جلول يعطيك ألف صحة و ربي يبارك فيك وكأنك شر...,1,سي جلول يعطيك ألف صحة و ربي يبارك فيك وكأنك شر...,,سي جلول يعطيك ألف صحة و ربي يبارك فيك وكأنك شر...,سي جلول يعطيك الف صحة و ربي يبارك فيك وكانك شر...,جلول يعطيك الف صحة ربي يبارك فيك وكانك شرهة شب...
7,59OI2D8,Ya mama mali masta lasta :p :p,0,Ya mama mali masta lasta :p :p,:p :p,Ya mama mali masta lasta,يا ماما مالي ماستا لاستا,ماما مالي ماستا لاستا
8,E1TVXKP,Bravo Neji jelloul,1,Bravo Neji jeloul,,Bravo Neji jeloul,براڤو نجي جلول,براڤو نجي جلول
9,L5L4SAC,نسحو يفهم في كل شيء,1,نسحو يفهم في كل شيء,,نسحو يفهم في كل شيء,نسحو يفهم في كل شيء,نسحو يفهم شيء


In [8]:
Emojis = list(df['emojis'].unique())


In [9]:
S=[]
for i in Emojis:
  for j in i.split():
    if j not in S: S+= [j,]

In [10]:
q=[]
for i in S :
  if len(i)>1:
    q+=[i,]
q

[':*', ':p', '<3', ':)', ':/', ':D', ':o', '=D', ':3', ':P', ';D', ':O']

In [11]:
D={
    ':*': u'😘' ,
    ':p': u'😜' ,
    '<3': u'💗' ,
    ':)': u'😊' ,
    ':/': u'😕' ,
    ':D': u'😃' ,
    ':o': u'😮',
    '=D': u'😃' ,
    ':3': u'😝' ,
    ':P': u'😜' ,
    ';D': u'😉' ,
    ':O': u'😮' ,
}

In [12]:
def replace_emoji(x):
  list=x.split(" ")
  M=[]
  for e in list:
    if e in D.keys():
      M.append(D[e])
    else:
      M.append(e)
  return ' '.join(M)
  

df['emojis']=df['emojis'].apply(replace_emoji)

In [13]:
df.head(50)

,ID,text,label,preprocessedtext,emojis,noemojitext,arabictext,NoStopWords
0,IQOTJAT,m3alem bourjilia w illi ma yefehmouch yelzmou ...,1,m3alem bourjilia w ili ma yefehmouch yelzmou y...,,m3alem bourjilia w ili ma yefehmouch yelzmou y...,معالم بورجيليا و يلي ما يفهموش يلزمو ياقرا ان...,معالم بورجيليا يلي يفهموش يلزمو ياقرا انس پلوس...
1,HY9M63D,Ya m3alllam,1,Ya m3alam,,Ya m3alam,يا معالام,معالام
2,7SV55S2,Ma7lek! Zin ou fannena 7loua.,1,Ma7lek! Zin ou fanena 7loua.,,Ma7lek! Zin ou fanena 7loua.,ماحلك زين و فاننا حلوا,ماحلك زين فاننا حلوا
3,KDLJVZR,hhhhh ya bliiiiiiiida ya Hanen,0,h ya blida ya Hanen,,h ya blida ya Hanen,ه يا بليدا يا هانن,بليدا هانن
4,0MAU5GY,Nikraha w ma5yebha pffff,0,Nikraha w ma5yebha pf,,Nikraha w ma5yebha pf,نيكراها و ماخيبها پف,نيكراها ماخيبها
5,EA1VCJ2,3sal denya :*,1,3sal denya :*,😘,3sal denya,عسال دنيا,عسال دنيا
6,2SZ11JC,سي جلول يعطيك ألف صحة و ربي يبارك فيك وكأنك شر...,1,سي جلول يعطيك ألف صحة و ربي يبارك فيك وكأنك شر...,,سي جلول يعطيك ألف صحة و ربي يبارك فيك وكأنك شر...,سي جلول يعطيك الف صحة و ربي يبارك فيك وكانك شر...,جلول يعطيك الف صحة ربي يبارك فيك وكانك شرهة شب...
7,59OI2D8,Ya mama mali masta lasta :p :p,0,Ya mama mali masta lasta :p :p,😜 😜,Ya mama mali masta lasta,يا ماما مالي ماستا لاستا,ماما مالي ماستا لاستا
8,E1TVXKP,Bravo Neji jelloul,1,Bravo Neji jeloul,,Bravo Neji jeloul,براڤو نجي جلول,براڤو نجي جلول
9,L5L4SAC,نسحو يفهم في كل شيء,1,نسحو يفهم في كل شيء,,نسحو يفهم في كل شيء,نسحو يفهم في كل شيء,نسحو يفهم شيء


In [14]:
!pip install emosent-py
!pip install emojis
import emojis
from emosent import get_emoji_sentiment_rank
def emoji_score(text):

  emos = emojis.get(text)

  if len(emos) !=0:

    score = 0

    for emo in emos:

      try:

        score += get_emoji_sentiment_rank(emo)["sentiment_score"]

      except:

        continue

    return score / len(emos)

  else:

    return 0


df['emoji_sentiment'] = df['emojis'].apply(lambda x : emoji_score(x)   )


  Created wheel for emosent-py: filename=emosent_py-0.1.6-cp36-none-any.whl size=28502 sha256=9ca31a6b7f87e482ea06067361dc018974934b820835530fd732112adb8dc3c1
  Stored in directory: /root/.cache/pip/wheels/46/32/3c/2e21c3622b77cdc89a38a711240588ac3cf9b8e805eed0f6e1
Successfully built emosent-py


In [15]:
df.head(50)

,ID,text,label,preprocessedtext,emojis,noemojitext,arabictext,NoStopWords,emoji_sentiment
0,IQOTJAT,m3alem bourjilia w illi ma yefehmouch yelzmou ...,1,m3alem bourjilia w ili ma yefehmouch yelzmou y...,,m3alem bourjilia w ili ma yefehmouch yelzmou y...,معالم بورجيليا و يلي ما يفهموش يلزمو ياقرا ان...,معالم بورجيليا يلي يفهموش يلزمو ياقرا انس پلوس...,0.000
1,HY9M63D,Ya m3alllam,1,Ya m3alam,,Ya m3alam,يا معالام,معالام,0.000
2,7SV55S2,Ma7lek! Zin ou fannena 7loua.,1,Ma7lek! Zin ou fanena 7loua.,,Ma7lek! Zin ou fanena 7loua.,ماحلك زين و فاننا حلوا,ماحلك زين فاننا حلوا,0.000
3,KDLJVZR,hhhhh ya bliiiiiiiida ya Hanen,0,h ya blida ya Hanen,,h ya blida ya Hanen,ه يا بليدا يا هانن,بليدا هانن,0.000
4,0MAU5GY,Nikraha w ma5yebha pffff,0,Nikraha w ma5yebha pf,,Nikraha w ma5yebha pf,نيكراها و ماخيبها پف,نيكراها ماخيبها,0.000
5,EA1VCJ2,3sal denya :*,1,3sal denya :*,😘,3sal denya,عسال دنيا,عسال دنيا,0.702
6,2SZ11JC,سي جلول يعطيك ألف صحة و ربي يبارك فيك وكأنك شر...,1,سي جلول يعطيك ألف صحة و ربي يبارك فيك وكأنك شر...,,سي جلول يعطيك ألف صحة و ربي يبارك فيك وكأنك شر...,سي جلول يعطيك الف صحة و ربي يبارك فيك وكانك شر...,جلول يعطيك الف صحة ربي يبارك فيك وكانك شرهة شب...,0.000
7,59OI2D8,Ya mama mali masta lasta :p :p,0,Ya mama mali masta lasta :p :p,😜 😜,Ya mama mali masta lasta,يا ماما مالي ماستا لاستا,ماما مالي ماستا لاستا,0.456
8,E1TVXKP,Bravo Neji jelloul,1,Bravo Neji jeloul,,Bravo Neji jeloul,براڤو نجي جلول,براڤو نجي جلول,0.000
9,L5L4SAC,نسحو يفهم في كل شيء,1,نسحو يفهم في كل شيء,,نسحو يفهم في كل شيء,نسحو يفهم في كل شيء,نسحو يفهم شيء,0.000


In [16]:
# Split the labeled data (df) into a smaller train and eval set for local scoring
train = df.sample(frac=0.7)
train_df = train[['NoStopWords', 'label']]
eval_df = df.loc[~df.ID.isin(train.ID.values)][['NoStopWords', 'label']]
train_df.shape, eval_df.shape

((5881, 2), (2520, 2))

# Modelling

Using simpletransformers - see https://github.com/ThilinaRajapakse/simpletransformers for docs.

In [ ]:
# # Setting up some logging (optional)
# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

# Specify some settings
args={'reprocess_input_data': True, 
      'overwrite_output_dir': True, 
      'num_train_epochs': 2}

# Create a ClassificationModel
model = ClassificationModel('bert', "asafaya/bert-base-arabic", args=args)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Some weights of the model checkpoint at asafaya/bert-base-arabic were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at asafaya/bert-base-ar

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


108 | INFO |  Training of bert model complete. Saved to outputs/.
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:851: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
108 | INFO |  Converting to features started. Cache is not used.


108 | INFO | {'mcc': 0.8920029668744196, 'tp': 1231, 'tn': 1153, 'fp': 62, 'fn': 74, 'eval_loss': 0.2483937656734183}


In [ ]:
result # Nice for interpretation

{'eval_loss': 0.2483937656734183,
 'fn': 74,
 'fp': 62,
 'mcc': 0.8920029668744196,
 'tn': 1153,
 'tp': 1231}

In [ ]:
# View the raw outputs
model_outputs

array([[-2.52539062,  2.78515625],
       [-3.23632812,  3.42578125],
       [ 3.2734375 , -3.4140625 ],
       ...,
       [ 2.65234375, -2.98046875],
       [ 3.25      , -3.41796875],
       [-3.28515625,  3.5234375 ]])

In [ ]:
# Convert to predicted class
pred_class = np.argmax(model_outputs, axis=-1)
pred_class

array([1, 1, 0, ..., 0, 0, 1])

In [ ]:
# Score 
f1_score(eval_df['label'], pred_class)

0.947652040030793

In [ ]:
# And accuracy for interest: 
accuracy_score(eval_df['label'], pred_class)

0.946031746031746

# Generating Submission File

We get the model preds and save them following the sample submission.

In [ ]:

df2 = pd.read_csv('Test.csv', lineterminator='\n')
df2['preprocessedtext'] = df2['text'].apply(lambda x: preProcessing(x))
df2['emojis']= df2['preprocessedtext'].apply(lambda x: extract_emoji(x))
df2['noemojitext'] = df2['preprocessedtext'].apply(lambda x: remove_emoji(x))
df2['arabictext'] = df2['noemojitext'].apply(lambda x: transliterate(x, source='tn', target='ar', universal=True))
df2['arabictext'] = df2['arabictext'].apply(lambda x: remove_punct(x))
df2['NoStopWords'] = df2['arabictext'].apply(NoStop)
pred_labels, test_model_outputs = model.predict(df2['NoStopWords'])


108 | INFO |  Converting to features started. Cache is not used.


In [ ]:
df2.head(30)

,ID,text,label,preprocessedtext,emojis,noemojitext,arabictext,NoStopWords
0,7I09CSF,m3alma berjouliya,0,m3alma berjouliya,,m3alma berjouliya,معالما برجولييا,معالما برجولييا
1,EO5QHN8,جميلة,0,جميلة,,جميلة,جميلة,جميلة
2,NYI236K,محلاها ومحلى قصة شعرها,0,محلاها ومحلى قصة شعرها,,محلاها ومحلى قصة شعرها,محلاها ومحلا قصة شعرها,محلاها ومحلا قصة شعرها
3,15973AH,تفففففوه عليكم وعلى خمجكم يا صقاط هذا مسلسل تج...,0,تفوه عليكم وعلى خمجكم يا صقاط هذا مسلسل تجيبو ...,,تفوه عليكم وعلى خمجكم يا صقاط هذا مسلسل تجيبو ...,تفوه عليكم وعلا خمجكم يا صقاط هذا مسلسل تجيبو ...,تفوه عليكم وعلا خمجكم صقاط مسلسل تجيبو شهر رمض...
4,9ZGKVAX,B1 wejiha,0,B1 wejiha,,B1 wejiha,ب وجيها,وجيها
5,A612HYO,"""شوكات انا اشك في انك تونسي _x000D_",0,"""شوكات انا اشك في انك تونسي _x0D_",,"""شوكات انا اشك في انك تونسي _x0D_",شوكات انا اشك في انك تونسي كزد,شوكات انا اشك انك تونسي كزد
6,U2AV11E,مزية مزية ماعادش تجيبهالنا ليلى الشابي راهي ام...,0,مزية مزية ماعادش تجيبهالنا ليلى الشابي راهي ام...,,مزية مزية ماعادش تجيبهالنا ليلى الشابي راهي ام...,مزية مزية ماعادش تجيبهالنا ليلا الشابي راهي ام...,مزية مزية ماعادش تجيبهالنا ليلا الشابي راهي ام...
7,FLAJ8DW,maset barcha kyarou fil radio w maya wini ? w ...,0,maset barcha kyarou fil radio w maya wini ? w ...,,maset barcha kyarou fil radio w maya wini ? w ...,ماست بارشا كيارو فيل راديو و مايا ويني ؟ و ريه...,ماست بارشا كيارو فيل راديو مايا ويني ريهم اليي...
8,CHQ7WMD,رجل جليل بصراحة ..ولو ابتعد على النهضة لكان أفضل,0,رجل جليل بصراحة .ولو ابتعد على النهضة لكان أفضل,,رجل جليل بصراحة .ولو ابتعد على النهضة لكان أفضل,رجل جليل بصراحة ولو ابتعد علا النهضة لكان افضل,رجل جليل بصراحة ابتعد علا النهضة لكان افضل
9,AX9M7WX,7lowa,0,7lowa,,7lowa,حلووا,حلووا


In [ ]:
test_pred_class = np.argmax(test_model_outputs, axis=-1)

In [ ]:
f1_score(pred_labels, np.array(df2["label"]))

0.0

In [ ]:
ss = pd.read_csv('SampleSubmission.csv')
ss.head()

,ID,label
0,7I09CSF,0
1,EO5QHN8,0
2,NYI236K,0
3,15973AH,0
4,9ZGKVAX,0


In [ ]:
ss['label'] = pred_labels
ss.head()

,ID,label
0,7I09CSF,1
1,EO5QHN8,1
2,NYI236K,1
3,15973AH,0
4,9ZGKVAX,1


In [ ]:
ss.to_csv('Sub1.csv', index=False)

# Good luck :)

by Johno Whitaker